In [1]:
import pandas as pd 
df = pd.read_csv('../data/stm_datasets/poliblogs2008.csv')
df['doc'] = df['documents']
df

,Unnamed: 0,documents,docname,rating,day,blog,doc
0,1,"After a week of false statements, lies, and di...",at0800300_1.text,Conservative,3,at,"After a week of false statements, lies, and di..."
1,2,I honestly don't know how either party's caucu...,at0800300_2.text,Conservative,3,at,I honestly don't know how either party's caucu...
2,3,While we stand in awe of the willingness of ou...,at0800300_3.text,Conservative,3,at,While we stand in awe of the willingness of ou...
3,4,These pages recently said goodbye to global wa...,at0800300_4.text,Conservative,3,at,These pages recently said goodbye to global wa...
4,5,A US report shows how the enemy controlled the...,at0800300_5.text,Conservative,3,at,A US report shows how the enemy controlled the...
...,...,...,...,...,...,...,...
13241,13242,Check out this new mailer -- forwarded our way...,tpm0800700_0.text,Liberal,7,tpm,Check out this new mailer -- forwarded our way...
13242,13243,Here's some detail on the internal discussions...,tpm0800700_1.text,Liberal,7,tpm,Here's some detail on the internal discussions...
13243,13244,Edwards senior adviser Joe Trippi has a theory...,tpm0800700_2.text,Liberal,7,tpm,Edwards senior adviser Joe Trippi has a theory...
13244,13245,This is interesting. The Huffington Post had a...,tpm0800800_0.text,Liberal,8,tpm,This is interesting. The Huffington Post had a...


In [2]:
import sys
sys.path.append('../gtm/')

from utils import text_processor
p = text_processor('en_core_web_sm', pos_tags_to_keep = ['VERB', 'NOUN'])
df['doc_clean'] = p.process_docs(df['doc'])

/home/germain/Documents/topic models/generalized_topic_model/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 13246/13246 [02:23<00:00, 92.49it/s]


In [3]:
from corpus import GTMCorpus

# Create a GTMCorpus object
train_dataset = GTMCorpus(
    df, 
    prevalence = "~ rating", # + C(speech_year) 
    content = "~ rating" # + C(speech_year)
)

train_dataset.M_prevalence_covariates.shape

(13246, 2)

In [4]:
from gtm import GTM

# Train the model
tm = GTM(
    train_dataset, 
    n_topics=20,
    doc_topic_prior='dirichlet', # logistic_normal
    alpha=0.1,
    prevalence_covariates_regularization=0.1,
    update_prior=True,
    encoder_hidden_layers=[], # structure of the encoder neural net
    decoder_hidden_layers=[300], # structure of the decoder neural net
    num_epochs=10,
    print_every=10000,
    log_every=1,
    w_prior=None,
    batch_size=250
)


Epoch   1	Mean Training Loss:27.9960805

Topic_0: ['help', 'election', 'run', 'support', 'time']
Topic_1: ['make', 'say', 'get', 'war', 'state']
Topic_2: ['get', 'go', 'war', 'way', 'take']
Topic_3: ['state', 'time', 'make', 'campaign', 'get']
Topic_4: ['look', 'go', 'day', 'think', 'take']
Topic_5: ['make', 'think', 'do', 'week', 'poll']
Topic_6: ['say', 'look', 'lead', 'run', 'win']
Topic_7: ['state', 'give', 'voter', 'look', 'do']
Topic_8: ['country', 'campaign', 'take', 'war', 'come']
Topic_9: ['go', 'show', 'say', 'vote', 'use']
Topic_10: ['get', 'look', 'hold', 'race', 'mean']
Topic_11: ['use', 'show', 'time', 'go', 'win']
Topic_12: ['get', 'vote', 'know', 'poll', 'believe']
Topic_13: ['take', 'get', 'day', 'tell', 'work']
Topic_14: ['year', 'way', 'show', 'go', 'country']
Topic_15: ['think', 'go', 'take', 'month', 'continue']
Topic_16: ['show', 'know', 'candidate', 'medium', 'say']
Topic_17: ['give', 'work', 'one', 'government', 'turn']
Topic_18: ['country', 'know', 'election',


Epoch   8	Mean Training Loss:4.4216963

Topic_0: ['video', 'update', 'post', 'link', 'reporter']
Topic_1: ['poll', 'campaign', 'race', 'margin', 'say']
Topic_2: ['torture', 'interrogation', 'waterboarding', 'administration', 'technique']
Topic_3: ['poll', 'campaign', 'race', 'margin', 'delegate']
Topic_4: ['http', 'org', 'scriptalreadyloade', 'src', 'swflash']
Topic_5: ['price', 'country', 'health', 'care', 'people']
Topic_6: ['poll', 'delegate', 'win', 'margin', 'primary']
Topic_7: ['price', 'oil', 'state', 'drilling', 'poll']
Topic_8: ['campaign', 'warming', 'report', 'today', 'ad']
Topic_9: ['official', 'police', 'kill', 'investigation', 'law']
Topic_10: ['mortgage', 'market', 'tax', 'lender', 'loan']
Topic_11: ['film', 'an', 'sura', 'qur', 'vv']
Topic_12: ['poll', 'margin', 'delegate', 'polling', 'lead']
Topic_13: ['update', 'say', 'video', 'think', 'reader']
Topic_14: ['poll', 'margin', 'delegate', 'race', 'win']
Topic_15: ['tax', 'taxis', 'mortgage', 'market', 'income']
Topic_16

In [5]:
# Assess the quality of the learned word embeddings 
# Top 8 closest words to a specific word

import torch
import torch.nn.functional as F

specific_word = 'tax'

word_id = [i for i,w in enumerate(train_dataset.vocab) if w == specific_word][0]

words = tm.AutoEncoder.decoder['dec_1'].weight.T

logit = torch.matmul(words.T[word_id], words)

beta = F.softmax(logit)

tm.AutoEncoder.eval()
topic_words = []
vals, indices = torch.topk(beta, 8)
vals = vals.cpu().tolist()
indices = indices.cpu().tolist()
[tm.id2token[idx] for idx in indices]

/tmp/ipykernel_41768/3978563657.py:15: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  beta = F.softmax(logit)


['tax', 'taxis', 'income', 'revenue', 'cut', 'rate', 'mortgage', 'spending']

In [6]:
dfc = tm.estimate_effect(train_dataset, n_samples=10, topic_ids=None)
dfc

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:44<00:00,  4.48s/it]


,topic,covariate,mean,sd
0,0,Intercept,0.023117,0.000263
1,0,rating[T.Liberal],-0.002721,0.000393
2,1,Intercept,0.034084,0.000195
3,1,rating[T.Liberal],0.001460,0.000227
4,2,Intercept,0.016312,0.000234
5,2,rating[T.Liberal],0.002677,0.000328
6,3,Intercept,0.050208,0.000274
7,3,rating[T.Liberal],-0.001972,0.000376
8,4,Intercept,0.044004,0.000302
9,4,rating[T.Liberal],-0.002694,0.000488


In [7]:
import statsmodels.api as sm
Y = tm.get_doc_topic_distribution(train_dataset)
X = train_dataset.M_prevalence_covariates
model = sm.OLS(Y[:,1],X)
results = model.fit()
covs = train_dataset.prevalence_colnames
pd.DataFrame([covs, results.params])

,0,1
0,Intercept,rating[T.Liberal]
1,0.03168,0.001144


In [8]:
tm.get_top_docs(train_dataset, topic_id = 10)

Topic: 10 | Document index: 11806 | Topic share: 0.9990015625953674
Hillary Clinton gave her big speech on the mortgage crisis today in Philadelphia, laying out various proposals for restructuring debts, an important issue to many blue-collar voters in this key primary â a state where a loss would effectively end her candidacy, and where even no less than a resounding win would be necessary for her to be credible for the nomination.  Beyond that, however, it was in many ways about laying out her establishment credibility with Pennsylvania voters â she was accompanied by such leading Pennsylvania figures as Gov. Ed Rendell and Philly Mayor Michael Nutter â and contrasting herself against Barack Obama on the experience issue, without ever naming him directly.  Key quote:  So we need a president who can restore our confidence, a president who is ready to confront complex economic problems with comprehensive solutions, a president who will act at the first signs of trouble, working w

In [9]:
tm.prior.prevalence_covariates_size

2

In [10]:
tm.prior.sample(N=X.shape[0], M_prevalence_covariates = X, epoch = 10)[0]

tensor([0.0106, 0.0614, 0.0332, 0.0328, 0.0286, 0.1058, 0.0185, 0.0300, 0.1164,
        0.0637, 0.0449, 0.0630, 0.0136, 0.0169, 0.0144, 0.1153, 0.0523, 0.0899,
        0.0393, 0.0493], device='cuda:0')